In [14]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import json

#게임 전적 검색 사이트
lol = "https://www.op.gg/"
overwatch = "https://overwatch.op.gg/"
battleground = "https://pubg.op.gg/"
suddenattack = "https://3rd.supply/"
maplestory = "https://maple.gg/"

#ele == elements
lol_ele = {
    "input_box" : "/html/body/div[2]/div[2]/form/input",
    "last_update" : "/html/body/div[2]/div[2]/div/div/div[1]/div[4]/span", 
    "ref_button" : "//*[@id='SummonerRefreshButton']"
}

game = input("전적 검색을 할 게임을 입력하세요\n검색어 : 롤, 오버워치, 배그, 서든어택, 메이플\n")

print(game, "을(를) 선택하셨습니다.\n", sep = '')

with webdriver.Chrome() as driver : 
    if(game == "롤") :
        driver.get(lol)
        driver.implicitly_wait(1)
        user = input("자녀분의 게임 아이디를 입력해주세요 :\n")
        
        #유저 아이디 입력
        #print(driver.find_element_by_xpath(lol_ele["input_box"]).is_enabled())
        driver.find_element_by_xpath(lol_ele["input_box"]).click()
        driver.find_element_by_xpath(lol_ele["input_box"]).send_keys(user)
        driver.find_element_by_xpath(lol_ele["input_box"]).send_keys(Keys.RETURN)
        driver.implicitly_wait(3)
        
        #전적 갱신
        #4. 에러 처리해야함
        lastupdate = driver.find_element_by_xpath(lol_ele["last_update"])
        #print(lastupdate.text)
        #몇초 전에 갱신한 기록은 경고알림만 뜸
        if(lastupdate != "몇초 전") : #만약 몇초 전에 갱신한 기록이 아니라면
            driver.find_element_by_xpath(lol_ele["ref_button"]).click()
            time.sleep(3)
            driver.implicitly_wait(3)
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        
        #최근 전적 (xxxx년 x월 x일 [오전,오후] x시 x분) 가져옴.
        test = soup.select("div.GameItemList > div:nth-child(1) > div > div.Content > div.GameStats > div.TimeStamp > span")
        print(test[0]['title'])
        
        record = dict(user = test[0]['title'])
        with open("GAMERECORD_LOL.json", "r", encoding='UTF8') as fp :
            json_data = json.load(fp)
            if not json_data[user] :
                json_data[user].append(test[0]['title'])
            else : 
                json_data[user] = test[0]['title']
                
        with open("GAMERECORD_LOL.json", "w", encoding='UTF8') as fp :
            json.dump(json_data, fp)
        
        #2-1. 이전 기록 저장해둔 파일 비교하기
        #2-2. 만약 처음 검사한 ID면 파일 만들기
        #2-2-1. 처음 만든 ID는 처음 만들었다고 하고 가장 마지막에 플레이한 경기 날짜 보여주기
        #3. 변화가 있는 경우와 없는 경우 해당 내용 출력하기

전적 검색을 할 게임을 입력하세요
검색어 : 롤, 오버워치, 배그, 서든어택, 메이플
롤
롤을(를) 선택하셨습니다.

자녀분의 게임 아이디를 입력해주세요 :
달려라포비돌격대


UnexpectedAlertPresentException: Alert Text: 71초 전에 갱신을 했습니다. 49초 후에 다시 갱신하실 수 있습니다.
Message: unexpected alert open: {Alert text : 71초 전에 갱신을 했습니다. 49초 후에 다시 갱신하실 수 있습니다.}
  (Session info: chrome=85.0.4183.121)
